In [1]:
import os
import pymysql
import pandas as pd
import matplotlib.pyplot as plt

host='localhost'
port='3306'
user='user001'
password='mysqluser001'
database='gbizinfo'

conn=pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')

In [2]:
datasets=pd.read_sql_query(
    "SELECT houjin_num,houjin_name,address FROM kihon where left(setsuritsu,4)='2000' && left(address,3)='東京都'",conn)

C:\Users\key\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
datasets

,houjin_num,houjin_name,address
0,1010001067375,イーコーポレーションドットジェーピー株式会社,東京都中央区日本橋大伝馬町１２番１号
1,1010001067920,株式会社ジェイ・ピー・システムズ,東京都千代田区平河町１丁目３番１３号平河町フロントビル１階
2,1010001070701,パーカッショネア・ジャパン株式会社,東京都港区芝浦２丁目３番３１号
3,1010001074941,シーアンドゼットコミュニケーション株式会社,東京都中央区銀座１丁目１６番地１号東貨ビル９階
4,1010001080436,株式会社日本不動産鑑定パートナーズ,東京都千代田区神田錦町２丁目７番地
...,...,...,...
382,9011501013415,株式会社デジタルソリューション,東京都北区上中里２丁目９番１号
383,9012401013439,デリバリーサービス株式会社,東京都府中市若松町３丁目２８番地の７
384,9012801003782,株式会社ニュウイング,東京都立川市上砂町５丁目７５番４号３０３
385,9013401001540,株式会社ファイネックス,東京都多摩市桜ヶ丘１丁目１６番地の６


In [4]:
import urllib
import urllib.request as request
import pandas as pd
import json
import time

url='https://msearch.gsi.go.jp/address-search/AddressSearch?q='
temp4=pd.DataFrame()

for i,n in enumerate(datasets.address):
    url='https://msearch.gsi.go.jp/address-search/AddressSearch?q='
    url2=url+urllib.parse.quote(n)

    try:
        response=request.urlopen(url2)
        contents=response.read()
        print(contents.decode())
        temp2=json.loads(contents.decode())
        temp3=pd.json_normalize(temp2)
        temp4=pd.concat([temp4,temp3],axis=0)
        response.close()
        time.sleep(3)
    except urllib.error.HTTPError:
        print("not available")
    else:
        pass

[{"geometry":{"coordinates":[139.780807,35.690079],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"東京都中央区日本橋大伝馬町１２番１号"}}]
[{"geometry":{"coordinates":[139.739792,35.683113],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"東京都千代田区平河町一丁目３番１３号"}}]
[{"geometry":{"coordinates":[139.754501,35.644085],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"東京都港区芝浦二丁目３番"}}]
[{"geometry":{"coordinates":[139.770599,35.674118],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"東京都中央区銀座一丁目１６番"}}]
[{"geometry":{"coordinates":[139.762909,35.691624],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"東京都千代田区神田錦町二丁目７番"}}]
[{"geometry":{"coordinates":[139.690186,35.636913],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"東京都目黒区五本木一丁目３３番１３号"}}]
[{"geometry":{"coordinates":[139.708603,35.689278],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"東京都新宿区

KeyboardInterrupt: 

In [5]:
latlng=pd.DataFrame()
for i,n in enumerate(temp4["geometry.coordinates"]):
    tmp=str(n).strip()
    tmp2=str(tmp).strip("[")
    tmp3=str(tmp2).strip("]")
    tmp4=tmp3.split(",")
    lat_tmp=float(tmp4[1])
    lng_tmp=float(tmp4[0])

    latlng_tmp=pd.concat([pd.Series(lat_tmp),pd.Series(lng_tmp)],axis=1)
    latlng=pd.concat([latlng,latlng_tmp],axis=0,ignore_index=True)

In [6]:
columns=["lat","lng"]
latlng.columns=columns
latlng

,lat,lng
0,35.690079,139.780807
1,35.683113,139.739792
2,35.644085,139.754501
3,35.674118,139.770599
4,35.691624,139.762909
...,...,...
192,35.665234,139.711334
193,35.649406,139.748093
194,35.638618,139.746078
195,35.671112,139.741013


In [12]:
import folium
import geocoder
from folium.plugins import HeatMap

location='東京駅'
address2=geocoder.osm(location,timeout=5.0)
address2.latlng

map2=folium.Map(location=address2.latlng,zoom_start=10)
HeatMap(latlng,radius=10,blur=5).add_to(map2)
map2

Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=%E6%9D%B1%E4%BA%AC%E9%A7%85&format=jsonv2&addressdetails=1&limit=1


In [11]:
lat = 35.6291112
lon = 139.7389313
 
map = folium.Map(location=[lat, lon], zoom_start=13)
map